## Visualize the outputs of the trained model

In [ ]:
# Import statements


from operator import add

from tqdm import tqdm
from dataset import get_data
from augmentaions import *
from models import *
from metrics import *
from utils import read_image, process_image
import random
import cv2
import numpy as np
import torch
from models import *
from glob import glob
import matplotlib.pyplot as plt
from dataset import *

In [ ]:
# Preambles

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
config = {"model": model,
          "data": data,
          "loss_fn": loss,
          "size": (224, 224),
          "augments": False,
          "data_path": '/mnt/qb/berens/users/jfadugba97/RetinaSegmentation/datasets/',
          "result_path": '/mnt/qb/berens/users/jfadugba97/RetinaSegmentation/results',
          "device": "cuda",
          "lr": 0.0001,
          "batch_size": 8,
          "epochs": 50,
          "num_workers": 2, 
          }

In [ ]:
# Load the data
_, _, test_x, test_y = get_data(config)

In [ ]:
predictor_name = f"{config['result_path']}/{config['model']}/{config['data']}/No_Augmentation_{config['loss_fn']}"


In [ ]:
models = [DeepLab, FPN, Unet]
model_names = ["DeepLab", "FPN", "Unet"]
# x_sub, y_sub = zip(*random.sample(list(zip(test_x, test_y)), 5))
data, label = zip(*random.sample(list(zip(test_x, test_y)), 5))
# print(len(x), len(y))

In [ ]:
def return_pred(x, model):
    with torch.no_grad():
        """ Prediction and Calculating FPS """
        pred_y = model.predict(x)

        pred_y = pred_y[0].cpu().numpy()  # (1, 512, 512)
        pred_y = np.squeeze(pred_y, axis=0)  # (512, 512)
        pred_y = pred_y > 0.5
        pred_y = np.array(pred_y, dtype=np.uint8)

    return pred_y

In [ ]:
def get_batch_predictions(x, model):
    """ get predictions """

    predictions = {}
    for (modelname, model_constructor) in zip(model_names, models):
        try:
            state_fname = f"results/No_Augmentation_{modelname}-{data_str}"
            model = model_constructor().to(device)
            model.load_state_dict(torch.load(state_fname, map_location=device))
        except FileNotFoundError:
            print(f"{state_fname} not found, continuing...")
            continue
        prediction = return_pred(x, model)
        predictions[modelname] = prediction


    return predictions

## Plotting images predictions for different output

In [ ]:
for i, (x, y) in enumerate(zip(data, label)):

    model_constructor = None
    """ Extract the name """
    name = x.split("/")[-1].split(".")[0]

    """ read and process image """
    image, mask = read_image(x, y)
    x, y = process_image(image, mask)
    "Get predictios"
    loss_names = ["cldica", "Dice"]
    predictions = {}
    for loss_name in loss_names:
        predictor_name = f"{config['result_path']}/{config['model']}/{config['data']}/No_Augmentation_{loss_name}"
        model = model_constructor().to(device)
        model.load_state_dict(torch.load(predictor_name, map_location=device))
        #call function to get prediction.
        prediction = return_pred(x, model)
        predictions[loss_name] = prediction
    
    """ Visualization """
    fig, axes = plt.subplots(
        nrows=1, ncols=2+len(predictions), figsize=(10, 5))
    axes[0].imshow(image)
    axes[0].title.set_text(f"Original Image")
    axes[0].axis("off")
    axes[1].imshow(mask_parse(mask))
    axes[1].title.set_text(f"Ground Truth")
    axes[1].axis("off")
    for i, (k, v) in enumerate(predictions.items()):
        print
        axes[i+2].imshow(mask_parse(predictions[k])*255, cmap="seismic")
        axes[i+2].title.set_text(f"{k} Prediction")
        axes[i+2].axis("off")

    # save image
    fig.savefig(f"results/{data_str}/{name}.png")



In [ ]:


for i, (x, y) in enumerate(zip(data, label)):

    """ Extract the name """
    name = x.split("/")[-1].split(".")[0]

    """ read and process image """
    image, mask = read_image(x, y)
    x, y = process_image(image, mask)

    predictions = {}
    for (modelname, model_constructor) in zip(model_names, models):
        try:
            state_fname = f"results/No_Augmentation_{modelname}-{data_str}"
            model = model_constructor().to(device)
            model.load_state_dict(torch.load(state_fname, map_location=device))
        except FileNotFoundError:
            print(f"{state_fname} not found, continuing...")
            continue
        prediction = return_pred(x, model)
        predictions[modelname] = prediction

    """ Visualization """
    fig, axes = plt.subplots(
        nrows=1, ncols=2+len(predictions), figsize=(10, 5))
    axes[0].imshow(image)
    axes[0].title.set_text(f"Original Image")
    axes[0].axis("off")
    axes[1].imshow(mask_parse(mask))
    axes[1].title.set_text(f"Ground Truth")
    axes[1].axis("off")
    for i, (k, v) in enumerate(predictions.items()):
        print
        axes[i+2].imshow(mask_parse(predictions[k])*255, cmap="seismic")
        axes[i+2].title.set_text(f"{k} Prediction")
        axes[i+2].axis("off")

    # save image
    fig.savefig(f"results/{data_str}/{name}.png")